In [15]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

%matplotlib inline
import pylab as plt
from pylab import rcParams

pickle_in = open("match_dataframe_processed.pickle","rb")
match_dataframe = pickle.load(pickle_in)

display(match_dataframe)

#Look at my player activity per month:
match_dataframe['Month'] = match_dataframe.Date.dt.month
match_dataframe.Date.dt.month.value_counts()

MonthGroup = match_dataframe.groupby('Month')
MonthGroup.Duration.describe()

#I don't like the time format, so I am going to convert duration to floats, since the most important part is
#the duration in minutes
match_dataframe['DurationMins'] = match_dataframe['Duration'].map(lambda x: round((x.total_seconds()/60), 2))

MonthGroup.DurationMins.describe()

#Looks like most of my games take around 40 minutes, 
#with some being as short as less than 1 minutes (probably disconnects),
#and some as long as 90 minutes.

#Let's vizualize some of these numbers:
match_dataframe['Month'].hist(bins=12);

#Now let's look at the distribution of played days in a month:
match_dataframe['Day'] = match_dataframe.Date.dt.day
match_dataframe.Date.dt.day.value_counts()

DayGroup = match_dataframe.groupby('Day')
DayGroup.DurationMins.describe()

#Seems like I play fairly equally on any day of the month!
#Let's see the histogram:
match_dataframe['Day'].hist(bins=31);

#What about weekdays?
match_dataframe['WeekDay'] = match_dataframe.Date.dt.weekday_name
match_dataframe.Date.dt.weekday_name.value_counts()

WeekDayGroup = match_dataframe.groupby('WeekDay')
WeekDayGroup.DurationMins.describe()

#No difference!
#Now let's visualize how many games per weekday I play:
match_dataframe['WeekDayNumber'] = match_dataframe.Date.dt.weekday
match_dataframe['WeekDayNumber'].hist(bins=7);

#What about the hour of the day? Note that the timestanp is given in UTC and needs to be converted:
match_dataframe['Hour'] = match_dataframe.Date.dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles').dt.hour
HourGroup = match_dataframe.groupby('Hour')

match_dataframe['Hour'].hist(bins=24);

#I mostly play in the evening, which makes sense.

#Now, I want to look at the entire time I played and see if there are any trends.
#I will create a separate dataframe for it:
timeline_analysis = pd.DataFrame(match_dataframe['Date'])

rcParams['figure.figsize'] = 50, 10
timeline_analysis.groupby([timeline_analysis['Date'].dt.year, timeline_analysis['Date'].dt.month]).count().plot(kind="bar");

#This graph explains a lot: I traveled in June-July 2014, had bad Internet in June-July 2015, went on a business trip
#in July 2016, traveled again in August 2017, and finally took this class in September-October 2017. 
#Also, I just graduated from college in 2014, so I had plenty of time to play, hence the high bars.

#However, my memory does not recall having any life events which would lead to the dip in activity late 2015/early 2016.
#That was a surprise to me! 